This part is a machine learning algorithm to recognise faces that figure in the training data

In [5]:
import os   # to create a training data set from images we have


In [6]:
Face_recognition_base= os.path.dirname(os.path.abspath('Face-Recognition.ipynb')) # define the path to our base
Face_recognition_images = os.path.join(Face_recognition_base,"Images")

In [11]:
# Create the training dataset

for root, dirs, images in os.walk(Face_recognition_images):
    for image in images :
        if image.endswith('png') or image.endswith('jpg') :
            path= os.path.join(root, image)
            label = os.path.basename(os.path.dirname(path))  # get labels that are the names of the directories
            # label = os.path.basename(root) // same as the previous line
# we need to make sure that each directory represents a label and it is an attached name (no spaces in it)
            
 
           